## HCL GWAS

Ce notebook Jupyter permet de semi-automatiser les études d'associations pangénomiques (GWAS: Genome Wide Assocation Studies), étape par étape.

**Contenu**:

1. Controle qualité des données de génotypage

2. Correction pour les effets de population

3. Analyse d'association

### Sections 

- [HCL GWAS notebook](#hcl-gwas)
- [Sections](#sections)
- [Requis](#requis)
- [Plink](#plink)
- [Contrôle Qualité](#contrôle-qualité)

### Requis

L'outil [Plink v1.9](https://www.cog-genomics.org/plink/) sera utilise pour la manipulation des donnees. Les graphes d'interpretation seront fait en Python avec [Matplotlib](#https://matplotlib.org/).

Versions des outils utilisés dans ce notebook:

 - plink v1.9
 - python v3.12
    - matplotlib 

### Plink

Plink est un outil de manipulation de données de génotypage qui utilise son propre format de fichier. Les génotypes sont représentés dans un ensemble de trois fichiers: le *.bim* contient les informations sur les variants, le *.fam* contient les informations sur les individus et le *.ped* les génotypes.

Le format des fichiers est détaillé ci-dessous:

<details>
<summary>Détails des formats plink</summary>

- **.bim**:

| Champ | Format |
| --- | --- |
| Chromosome code | int or X/Y/XY/MT (0 indicates unknown) |
| Variant ID | eg: rsID |
| Position in morgans or centimorgans | 0 = dummy value |
| Base-pair coordinates | 1-based, maximum is 2^31 - 2 |
| Allele 1 | Corresponding to the clear bits in the .ped file |
| Allele 2 | Corresponding to the set bits in the .ped file |

- **.fam**:

| Champ | Format |
| --- | --- |
| Family ID | / |
| Within-Family ID | Cannot be 0 |
| Within-Family ID or farther | 0 if not present in dataset |
| Within-Family ID or mother | 0 if not present in dataset |
| Sex code | 1 = male, 2 = female, 0 = unknown |
| Phenotype value | 1 = control, 2 = case, 0/-9/non-numeric = missing data |

- **.ped**:

| Code | Format |
| --- | --- |
| 00 | Homozygous for the 1st allele in the .bim file |
| 01 | Mising genotype |
| 10 | Heterozygous |
| 11 | Homozygous for the 2nd allele in the .bim file |

</details>

*note*: on utilisera surtout le *.bed*, la version binaire du *.ped*.

Plink est assez sensible aux identifiants dupliqués pour les variants, il est courant d'avoir des "." pour les rsids manquants. Il est conseillé de transformer ces IDs manquants sous un autre format, eg: *chr_pos_ref_alt*.

### Contrôle Qualité

##### 1a. Préparation du jeu de donnees:

Le jeu de donnees est un subset du [1000 Genomes Project](https://www.internationalgenome.org/).

In [ ]:
import subprocess
import os

wdir = "/home/corentin/Documents/GitHub/HCL-GWAS/test_dataset/"
os.chdir(wdir)

wget_vcf = subprocess.Popen(["wget", "https://storage.googleapis.com/hail-tutorial/1kg.vcf.bgz"],
                            stdout = subprocess.PIPE)
wget_out = wget_vcf.communicate()[0]

wget_annot = subprocess.Popen(["wget", "https://storage.googleapis.com/hail-tutorial/1kg_annotations.txt"],
                              stdout = subprocess.PIPE)
wget_out2 = wget_annot.communicate()[0]

### Optionnel: convertir un VCF en plink

Cette commande va convertir le fichier VCF en:
 - <bfile_prefix>.bed
 - <bfile_prefix>.bim
 - <bfile_prefix>.fam

<bfile_prefix>.nosex indique quels echantillons ont l'information sur le sexe manquant dans le fichier *.fam* (ie: tous les echantillons car l'information est absente du VCF).

In [ ]:
plink = "plink1.9"
bfile_prefix = "Caffeine"
vcf = "1kg.vcf.bgz"

vcf2plink = subprocess.Popen([plink,
                              "--vcf", vcf,
                              "--make-bed",
                              "--out", bfile_prefix],
                              stdout = subprocess.PIPE)

vcf2plink_out = vcf2plink.communicate()[0]

##### 1b. Filtre sur les données manquantes:

Contrôle qualité des données de genotypage. On retire les individus et les variants avec plus de 5% de genotypes manquants (**--mind** et **--geno**). Et produire l'Analyse en Composantes Principales (**--pca**).

In [ ]:
plink_qc = subprocess.Popen([plink, 
                            "--bfile", bfile_prefix,
                            "--geno", "0.05",
                            "-mind", "0.05",
                            "--make-bed",
                            "--pca",
                            "--out", bfile_prefix+"_QC"],
                            stdout = subprocess.PIPE)

plink_qc_output = plink_qc.communicate()[0]